Methods to generate seq id/ surrogate key in pyspark
1) monotonically_increasing_id()
2) row_number()- window function
3) crc32
4) md5
5) sha1 & sha2

In [0]:
df = spark.read.csv(path='dbfs:/FileStore/data/empCompanyData.csv', header=True, inferSchema=True, nullValue='null').drop_duplicates(['EMPNO']).dropna(how='all')

In [0]:
display(df)
df.printSchema()

EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO,UPDATED_DATE
1234,SEKHAR,doctor,7777,null,667,78,80,2022-01-03
7369,SMITH,CLERK,7902,1980-12-17,800,null,20,2022-01-04
7499,ALLEN,SALESMAN,7698,1981-02-20,1600,300,30,2022-01-04
7521,WARD,SALESMAN,7698,1981-02-22,1250,500,30,2022-01-04
7566,JONES,MANAGER,7839,1981-02-04,2975,null,20,2022-01-05
7654,MARTIN,SALESMAN,7698,1981-09-21,1250,1400,30,2022-01-03
7698,SGR,MANAGER,7839,1981-01-05,2850,null,30,2022-01-04
7782,RAVI,MANAGER,7839,1981-09-06,2450,null,10,2022-01-02
7788,SCOTT,ANALYST,7566,1987-04-19,3000,null,20,2022-01-02
7839,KING,PRESIDENT,null,1981-11-01,5000,null,10,2022-01-02


root
 |-- EMPNO: integer (nullable = true)
 |-- ENAME: string (nullable = true)
 |-- JOB: string (nullable = true)
 |-- MGR: integer (nullable = true)
 |-- HIREDATE: date (nullable = true)
 |-- SAL: integer (nullable = true)
 |-- COMM: integer (nullable = true)
 |-- DEPTNO: integer (nullable = true)
 |-- UPDATED_DATE: date (nullable = true)



In [0]:
#using monotonically_increasing_id() function
from pyspark.sql.functions import monotonically_increasing_id, row_number, lit, crc32, col, md5, sha2
from pyspark.sql.window import Window

In [0]:
df = df.withColumn('id', monotonically_increasing_id()+1)
display(df)

EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO,UPDATED_DATE,id
1234,SEKHAR,doctor,7777,null,667,78,80,2022-01-03,1
7369,SMITH,CLERK,7902,1980-12-17,800,null,20,2022-01-04,2
7499,ALLEN,SALESMAN,7698,1981-02-20,1600,300,30,2022-01-04,3
7521,WARD,SALESMAN,7698,1981-02-22,1250,500,30,2022-01-04,4
7566,JONES,MANAGER,7839,1981-02-04,2975,null,20,2022-01-05,5
7654,MARTIN,SALESMAN,7698,1981-09-21,1250,1400,30,2022-01-03,6
7698,SGR,MANAGER,7839,1981-01-05,2850,null,30,2022-01-04,7
7782,RAVI,MANAGER,7839,1981-09-06,2450,null,10,2022-01-02,8
7788,SCOTT,ANALYST,7566,1987-04-19,3000,null,20,2022-01-02,9
7839,KING,PRESIDENT,null,1981-11-01,5000,null,10,2022-01-02,10


In [0]:
# using row_number()- window function
df.withColumn('row_number', row_number().over(Window.partitionBy(lit('')).orderBy(lit('')))).show()

+-----+------+---------+----+----------+----+----+------+------------+---+----------+
|EMPNO| ENAME|      JOB| MGR|  HIREDATE| SAL|COMM|DEPTNO|UPDATED_DATE| id|row_number|
+-----+------+---------+----+----------+----+----+------+------------+---+----------+
| 1234|SEKHAR|   doctor|7777|      null| 667|  78|    80|  2022-01-03|  1|         1|
| 7369| SMITH|    CLERK|7902|1980-12-17| 800|null|    20|  2022-01-04|  2|         2|
| 7499| ALLEN| SALESMAN|7698|1981-02-20|1600| 300|    30|  2022-01-04|  3|         3|
| 7521|  WARD| SALESMAN|7698|1981-02-22|1250| 500|    30|  2022-01-04|  4|         4|
| 7566| JONES|  MANAGER|7839|1981-02-04|2975|null|    20|  2022-01-05|  5|         5|
| 7654|MARTIN| SALESMAN|7698|1981-09-21|1250|1400|    30|  2022-01-03|  6|         6|
| 7698|   SGR|  MANAGER|7839|1981-01-05|2850|null|    30|  2022-01-04|  7|         7|
| 7782|  RAVI|  MANAGER|7839|1981-09-06|2450|null|    10|  2022-01-02|  8|         8|
| 7788| SCOTT|  ANALYST|7566|1987-04-19|3000|null|    

In [0]:
# crc32 method- returns random numbers, it can generate duplicates after every 100k or 200k records
df.withColumn('crc32_key', crc32(col('EMPNO').cast('string'))).show()

+-----+------+---------+----+----------+----+----+------+------------+---+----------+
|EMPNO| ENAME|      JOB| MGR|  HIREDATE| SAL|COMM|DEPTNO|UPDATED_DATE| id| crc32_key|
+-----+------+---------+----+----------+----+----+------+------------+---+----------+
| 1234|SEKHAR|   doctor|7777|      null| 667|  78|    80|  2022-01-03|  1|2615402659|
| 7369| SMITH|    CLERK|7902|1980-12-17| 800|null|    20|  2022-01-04|  2|3163315632|
| 7499| ALLEN| SALESMAN|7698|1981-02-20|1600| 300|    30|  2022-01-04|  3|1046173690|
| 7521|  WARD| SALESMAN|7698|1981-02-22|1250| 500|    30|  2022-01-04|  4|3535170612|
| 7566| JONES|  MANAGER|7839|1981-02-04|2975|null|    20|  2022-01-05|  5| 683555987|
| 7654|MARTIN| SALESMAN|7698|1981-09-21|1250|1400|    30|  2022-01-03|  6|4024152101|
| 7698|   SGR|  MANAGER|7839|1981-01-05|2850|null|    30|  2022-01-04|  7|1255715586|
| 7782|  RAVI|  MANAGER|7839|1981-09-06|2450|null|    10|  2022-01-02|  8|3000238442|
| 7788| SCOTT|  ANALYST|7566|1987-04-19|3000|null|    

In [0]:
# md5- return 32 bit hash key based in EMPNO, not suggested on more than 50 million of records
df.withColumn('md5_key', md5(col('EMPNO').cast('string'))).show(truncate=False)

+-----+------+---------+----+----------+----+----+------+------------+---+--------------------------------+
|EMPNO|ENAME |JOB      |MGR |HIREDATE  |SAL |COMM|DEPTNO|UPDATED_DATE|id |md5_key                         |
+-----+------+---------+----+----------+----+----+------+------------+---+--------------------------------+
|1234 |SEKHAR|doctor   |7777|null      |667 |78  |80    |2022-01-03  |1  |81dc9bdb52d04dc20036dbd8313ed055|
|7369 |SMITH |CLERK    |7902|1980-12-17|800 |null|20    |2022-01-04  |2  |0d7f9017fbda691900187b22404b8a1f|
|7499 |ALLEN |SALESMAN |7698|1981-02-20|1600|300 |30    |2022-01-04  |3  |7a2b33c672ce223b2aa5789171ddde2f|
|7521 |WARD  |SALESMAN |7698|1981-02-22|1250|500 |30    |2022-01-04  |4  |e1e1f667ce4596e5644be6fab627c226|
|7566 |JONES |MANAGER  |7839|1981-02-04|2975|null|20    |2022-01-05  |5  |b937384a573b94c4d7cc6004c496f919|
|7654 |MARTIN|SALESMAN |7698|1981-09-21|1250|1400|30    |2022-01-03  |6  |e2a7555f7cabd6e31aef45cb8cda4999|
|7698 |SGR   |MANAGER  |7839

In [0]:
#sha2- applied on records more than 50/100/200 million. returns 256 bit hash key value
df = df.withColumn('sha2_id', sha2(col('EMPNO').cast('string'),256))
df.show(truncate=False)

+-----+------+---------+----+----------+----+----+------+------------+---+----------------------------------------------------------------+
|EMPNO|ENAME |JOB      |MGR |HIREDATE  |SAL |COMM|DEPTNO|UPDATED_DATE|id |sha2_id                                                         |
+-----+------+---------+----+----------+----+----+------+------------+---+----------------------------------------------------------------+
|1234 |SEKHAR|doctor   |7777|null      |667 |78  |80    |2022-01-03  |1  |03ac674216f3e15c761ee1a5e255f067953623c8b388b4459e13f978d7c846f4|
|7369 |SMITH |CLERK    |7902|1980-12-17|800 |null|20    |2022-01-04  |2  |c59f438f16c5a409eb2a040b299e82de37503321b9cbfec4fb351547261dd1b1|
|7499 |ALLEN |SALESMAN |7698|1981-02-20|1600|300 |30    |2022-01-04  |3  |4427dc2e32a1d099dbe2e3c093a8726e0ea72b9422c36a99915dae1d31e9385f|
|7521 |WARD  |SALESMAN |7698|1981-02-22|1250|500 |30    |2022-01-04  |4  |74ed8ca63e8b4fb8b8ac06e8df400f098b0b09cf1b89c8a331e72e1919b57bd4|
|7566 |JONES |MANAGE

In [0]:
%sql
select sha2('1234', 256)

"sha2(1234, 256)"
03ac674216f3e15c761ee1a5e255f067953623c8b388b4459e13f978d7c846f4
